# XArrayDataset

In [78]:
import autoroot
import typing as tp
from dataclasses import dataclass
import numpy as np
import pandas as pd
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDABatcher

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This tutorial walks through some of the nice features of the custom `XRDABatcher` class.
This is a custom class that slices and dices through an `xr.DataArray` where a user can specify explicitly the patch dimensions and the strides.
We preallocated the *slices* and then we can arbitrarily call the slices at will.
This is very similar to the *torch.utils.data* object except we are only working with `xr.DataArray`'s directly.


There have been other previous attempts at this, e.g. `xBatcher`.
However, we found the API very cumbersome and non-intuitive.
This is our attempt to design an API that we are comfortable with and that we find easy to use.

Below, we have outlined a few use-cases that users may be interested in. 
These use cases are:

* Chunking a 1-Dimensional Time Series
* Patch-ify a 2D Grid
* Cube-ify a 3D Volume
* Cube-ify a 2D+T Spatio-Temporal Field
* Reconstructing Multiple Variables
* Choosing Specific Dimensions for Reconstructions

We will walk through each of these and highlight how this can be achieved with the custom `XRDABatcher` class.

## Case I: Chunking a 1D TS

In [88]:
T = tp.Literal["t"]


@dataclass
class Variable1D:
    data: xrdataclass.Data[T, np.ndarray]
    t: xrdataclass.Coord[T, np.ndarray] = 0
    name: xrdataclass.Name[str] = "var"

In [90]:

t = np.arange(1, 360+1, 1)
rng = np.random.RandomState(seed=123)
ts = np.sin(t)

ts = Variable1D(data=ts, t=t, name="var")

da = xrdataclass.asdataarray(ts)

da

<xarray.DataArray 'var' (t: 360)>
array([0.8414709848078965, 0.9092974268256817, 0.1411200080598672,
       -0.7568024953079283, -0.9589242746631385, -0.27941549819892586,
       0.6569865987187891, 0.9893582466233818, 0.4121184852417566,
       -0.5440211108893698, -0.9999902065507035, -0.5365729180004349,
       0.4201670368266409, 0.9906073556948704, 0.6502878401571168,
       -0.2879033166650653, -0.9613974918795568, -0.7509872467716762,
       0.14987720966295234, 0.9129452507276277, 0.836655638536056,
       -0.008851309290403876, -0.8462204041751706, -0.9055783620066239,
       -0.13235175009777303, 0.7625584504796027, 0.956375928404503,
       0.27090578830786904, -0.6636338842129675, -0.9880316240928618,
       -0.404037645323065, 0.5514266812416906, 0.9999118601072672,
       0.5290826861200238, -0.428182669496151, -0.9917788534431158,
       -0.6435381333569995, 0.2963685787093853, 0.9637953862840878,
       0.7451131604793488, -0.158622668804709, -0.9165215479156338,
       -0.8317747426285983, 0.017701925105413577, 0.8509035245341184,
       0.9017883476488092, 0.123573122745224, -0.7682546613236668,
       -0.9537526527594719, -0.26237485370392877, 0.6702291758433747,
       0.9866275920404853, 0.39592515018183416, -0.5587890488516163,
       -0.9997551733586199, -0.5215510020869119, 0.43616475524782494,
       0.9928726480845371, 0.6367380071391379, -0.3048106211022167,
...
       0.6702068037805061, -0.2624039418616639, -0.9537617134939987,
       -0.7682353642374472, 0.12360303600011291, 0.9018013749637745,
       0.8508876886558596, 0.01767178546737087, -0.8317914757822045,
       -0.9165094902005468, -0.15859290602857282, 0.7451332645574127,
       0.9637873480674221, 0.2963397884973224, -0.6435612059762619,
       -0.9917749956098326, -0.42815542808445156, 0.5291082654818533,
       0.9999122598719259, 0.551401533867395, -0.4040652194563607,
       -0.9880362734541701, -0.6636113342009432, 0.2709348053161655,
       0.9563847343054627, 0.7625389491684939, -0.13238162920545193,
       -0.9055911481970673, -0.8462043418838514, -0.008821166113885877,
       0.8366721491002945, 0.9129329489429682, 0.14984740573347818,
       -0.7510071512506543, -0.9613891968218607, -0.2878744485084861,
       0.6503107401625525, 0.9906032333897737, 0.42013968223930687,
       -0.5365983551885637, -0.9999903395061709, -0.5439958173735323,
       0.412145950487085, 0.9893626321783087, 0.6569638725243397,
       -0.27944444178438205, -0.9589328250406132, -0.7567827912998033,
       0.14114985067939137, 0.9093099708898409, 0.8414546973619527,
       -3.014435335948845e-05, -0.8414872714892108, -0.9092848819352602,
       -0.14109016531210986, 0.7568221986283603, 0.9589157234143065],
      dtype=object)
Coordinates:
  * t        (t) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360

In this first example, we are going to do a non-overlapping style.
We will take a 30 day window with a 30 day stride.
This will give us exactly 12 patches (like 12 months).

In [93]:
patches = {"t": 30}
strides = {"t": 30}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")


XArray Patcher
DataArray size: OrderedDict([('t', 360)])
Patches:        OrderedDict([('t', 30)])
Strides:        OrderedDict([('t', 30)])
Num Batches:    OrderedDict([('t', 12)])
Dataset(size): 12


In this example, we will incorporate overlapping windows.
We will do a 30 day window but we will have a 15 day stride.
So, we have a 15 day overlap when creating the patches.
We can do the mental calculation already because it's quite simple:

$$
\text{Patches} = \frac{360 \text{ days total } - 30 \text{ day patches }}{15 \text{ day stride }} + 1
$$

If this is nicely divisible, we wont have any problems. 
However, often times it's not so we might have to use the `floor` operator to ensure we get integers
Our method will give a warning (optional) which lets the user know there is an issue.

In [96]:
patches = {"t": 30}
strides = {"t": 15}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('t', 360)])
Patches:        OrderedDict([('t', 30)])
Strides:        OrderedDict([('t', 15)])
Num Batches:    OrderedDict([('t', 23)])
Dataset(size): 23


## Case II: Patchify a 2D Grid

In [116]:
X = tp.Literal["x"]
Y = tp.Literal["y"]

@dataclass
class Variable2D:
    data: xrdataclass.Data[tuple[X, Y], np.ndarray]
    x: xrdataclass.Coord[X, np.ndarray] = 0
    y: xrdataclass.Coord[Y, np.ndarray] = 0
    name: xrdataclass.Name[str] = "var"

In [121]:

x = np.linspace(-1, 1, 128)
y = np.linspace(-2, 2, 128)
rng = np.random.RandomState(seed=123)

data = rng.randn(x.shape[0], y.shape[0])

grid = Variable2D(data=data, x=x, y=y, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray 'var' (x: 128, y: 128)>
array([[-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
        ..., -0.2702323920091581, 0.8123413299768204,
        0.49974014486517293],
       [0.4743472978718003, -0.5639239315264725, -0.9973214687759007,
        ..., -1.692052415377956, 0.7426862211616183, -0.0805832220429508],
       [0.5907042178943853, 0.11529872221079104, 0.029642925120495232,
        ..., -1.007508801235176, -1.3057858663416402,
        -0.8828289877220372],
       ...,
       [0.29171230394468933, 0.2600952551077316, 0.002036450475535042,
        ..., -0.11816623640879106, 1.0585932787669152,
        1.4154491223144148],
       [-1.1583491721325236, -0.11546117709313128, -2.3730459483526154,
        ..., -0.32826455571525703, 0.3578459393077905,
        1.2331876407716642],
       [-0.02535733785166554, -0.433411141285872, 1.0400184091258702,
        ..., 1.2995391133658956, 1.3579829817539886, -0.5473255221542979]],
      dtype=object)
Coordinates:
  * x        (x) object -1.0 -0.984251968503937 ... 0.984251968503937 1.0
  * y        (y) object -2.0 -1.968503937007874 ... 1.968503937007874 2.0

We will have a `[20,20]` patch with no overlap, `[20,20]`

In [122]:
patches = {"x": 8, "y": 8}
strides = {"x": 8, "y": 8}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8)])
Strides:        OrderedDict([('x', 8), ('y', 8)])
Num Batches:    OrderedDict([('x', 16), ('y', 16)])
Dataset(size): 256


We will have a `[20,20]` patch with some overlap, like the boundaries of 2, `[2,2]`

In [123]:
patches = {"x": 8, "y": 8}
strides = {"x": 2, "y": 2}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8)])
Strides:        OrderedDict([('x', 2), ('y', 2)])
Num Batches:    OrderedDict([('x', 61), ('y', 61)])
Dataset(size): 3721


## Case III: Cube-ify a 3D Volume

In [125]:
X = tp.Literal["x"]
Y = tp.Literal["y"]
Z = tp.Literal["z"]

@dataclass
class Variable3D:
    data: xrdataclass.Data[tuple[X, Y, Z], np.ndarray]
    x: xrdataclass.Coord[X, np.ndarray] = 0
    y: xrdataclass.Coord[Y, np.ndarray] = 0
    z: xrdataclass.Coord[Z, np.ndarray] = 0
    name: xrdataclass.Name[str] = "var"

In [126]:

x = np.linspace(-1, 1, 128)
y = np.linspace(-2, 2, 128)
z = np.linspace(-5, 5, 128)
rng = np.random.RandomState(seed=123)

data = rng.randn(x.shape[0], y.shape[0], z.shape[0])

grid = Variable3D(data=data, x=x, y=y, z=z, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray 'var' (x: 128, y: 128, z: 128)>
array([[[-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
         ..., -0.2702323920091581, 0.8123413299768204,
         0.49974014486517293],
        [0.4743472978718003, -0.5639239315264725, -0.9973214687759007,
         ..., -1.692052415377956, 0.7426862211616183,
         -0.0805832220429508],
        [0.5907042178943853, 0.11529872221079104, 0.029642925120495232,
         ..., -1.007508801235176, -1.3057858663416402,
         -0.8828289877220372],
        ...,
        [0.29171230394468933, 0.2600952551077316, 0.002036450475535042,
         ..., -0.11816623640879106, 1.0585932787669152,
         1.4154491223144148],
        [-1.1583491721325236, -0.11546117709313128, -2.3730459483526154,
         ..., -0.32826455571525703, 0.3578459393077905,
         1.2331876407716642],
        [-0.02535733785166554, -0.433411141285872, 1.0400184091258702,
         ..., 1.2995391133658956, 1.3579829817539886,
         -0.5473255221542979]],

...

       [[-0.6881305123337642, -0.31613607789675885, -1.1748570996384167,
         ..., 0.11708834945120478, -0.25029278772867203,
         0.006582839226798585],
        [-0.8521089756266904, 0.3789526844938861, -0.07201126022921452,
         ..., -0.26478353615363875, 0.0896649360782256,
         -0.24070543566603203],
        [-1.0979671486319609, 1.591515226944711, -1.3385513403063525,
         ..., 0.2124612698031788, 0.007847492953994804,
         1.631406119020994],
        ...,
        [0.18093950727860766, -0.7213022976611424, -0.9051824992175953,
         ..., 1.472395550025634, 0.5725734456205965,
         -1.6922121006486017],
        [-0.12132770780306669, 1.6944560110970008, 0.21141362344288625,
         ..., -1.155670654547361, -0.3809033093046882,
         1.1471051721305434],
        [0.07145388475934611, 0.14395917803585095, -1.500919698739799,
         ..., 1.6116897302953097, 0.13778696175786584,
         0.9260894932302164]]], dtype=object)
Coordinates:
  * x        (x) object -1.0 -0.984251968503937 ... 0.984251968503937 1.0
  * y        (y) object -2.0 -1.968503937007874 ... 1.968503937007874 2.0
  * z        (z) object -5.0 -4.921259842519685 ... 4.921259842519685 5.0

We will have a `[20,20]` patch with no overlap, `[20,20]`

In [127]:
patches = {"x": 8, "y": 8, "z": 8}
strides = {"x": 8, "y": 8, "z": 8}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128), ('z', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8), ('z', 8)])
Strides:        OrderedDict([('x', 8), ('y', 8), ('z', 8)])
Num Batches:    OrderedDict([('x', 16), ('y', 16), ('z', 16)])
Dataset(size): 4096


We will have a `[20,20]` patch with some overlap, like the boundaries of 2, `[2,2]`

In [ ]:
patches = {"x": 8, "y": 8, "z": 8}
strides = {"x": 2, "y": 2, "z": 2}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8)])
Strides:        OrderedDict([('x', 2), ('y', 2)])
Num Batches:    OrderedDict([('x', 61), ('y', 61)])
Dataset(size): 3721


## Case IV: Cube-ify a 2D+T Spatio-Temporal Field

In [129]:
X = tp.Literal["x"]
Y = tp.Literal["y"]
T = tp.Literal["t"]

@dataclass
class Variable2DT:
    data: xrdataclass.Data[tuple[T, X, Y], np.ndarray]
    t: xrdataclass.Coord[T, np.ndarray] = 0
    x: xrdataclass.Coord[X, np.ndarray] = 0
    y: xrdataclass.Coord[Y, np.ndarray] = 0
    name: xrdataclass.Name[str] = "var"

In [134]:

x = np.linspace(-1, 1, 200)
y = np.linspace(-2, 2, 200)
t = np.arange(1, 360+1, 1)
rng = np.random.RandomState(seed=123)

data = rng.randn(t.shape[0], x.shape[0], y.shape[0])

grid = Variable2DT(data=data, x=x, y=y, t=t, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray 'var' (t: 360, x: 200, y: 200)>
array([[[-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
         ..., -0.110850721053007, -0.3412617162065858,
         -0.21794626210294177],
        [0.7033101178067301, -0.5981053313408254, 2.200702099217128,
         ..., 0.2453794072596918, -0.16842643163047274,
         0.24402693814049461],
        [1.5340902893038646, -0.5299140986336766, -0.49097228256584685,
         ..., -1.3943751127131896, 0.9126879358198553,
         -1.273570236812244],
        ...,
        [-1.2542969690152004, -0.15342524305088331, 0.39214351549990845,
         ..., 1.491465851261915, -0.1372224191993455,
         -0.9896969060401962],
        [0.9222701981786514, 0.17628323295315496, 1.4639886215415012,
         ..., 1.3550973544616685, 1.2875271692153203,
         0.5203795117981033],
        [-0.03978635340989313, -0.18871495988964238, -1.539713800876724,
         ..., 0.525623317068983, 0.9948001649427021,
         -0.8035127297933919]],

...

       [[0.7102262472552882, -0.5875607673206947, 0.4671678722856565,
         ..., -1.9052417770209618, -0.06033504734222561,
         1.0841711687854474],
        [0.2502968527892984, -1.0295818080815926, 1.0811170953061782,
         ..., 0.48893107092405547, -0.39333678936911265,
         1.3902658557598195],
        [0.3809180829500794, -0.6194011859666362, 1.0491152834277517,
         ..., -1.4824745267577444, -0.2576864115553968,
         0.15616394970614753],
        ...,
        [0.041549417801490664, -1.0582280587825275, -0.9171024694729889,
         ..., 0.27116961538032347, -0.9032749318982085,
         0.0066784126903923135],
        [-1.1869855268998732, 1.485651170518838, 0.5748461712503058,
         ..., 0.8499812621585755, -0.6654115467365482,
         -0.7483549750354092],
        [-0.3397402215142891, -0.8361204555128399, -0.5343129072201397,
         ..., 0.8719576160884512, 0.19112131585959607,
         -0.3649096445758529]]], dtype=object)
Coordinates:
  * t        (t) object 1 2 3 4 5 6 7 8 9 ... 353 354 355 356 357 358 359 360
  * x        (x) object -1.0 -0.9899497487437185 ... 0.9899497487437185 1.0
  * y        (y) object -2.0 -1.979899497487437 ... 1.979899497487437 2.0

Now, this is a rather big field.
Let's say we want to use some ML method with a CNN to learn how to predict ...
However, ingesting this large patch would be very difficult.
So instead, we will use the standard size for many CNNs, which is `[64,64]`.
In addition, we will use a temporal window of 15 days. 
So the patch will be `[15,64,64]`. 

As will the above examples, we will also account for the overlap in the spatial borders with `[4,4]` strides.
And lastly, we will have a 5 day overlap for the time steps.
So the final strides will be `[5,4,4]`

In [135]:
patches = {"x": 64, "y": 64, "t": 15}
strides = {"x": 4, "y": 4, "t": 5}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('t', 360), ('x', 200), ('y', 200)])
Patches:        OrderedDict([('t', 15), ('x', 64), ('y', 64)])
Strides:        OrderedDict([('t', 5), ('x', 4), ('y', 4)])
Num Batches:    OrderedDict([('t', 70), ('x', 35), ('y', 35)])
Dataset(size): 85750


All of the sudden, we have a LOT of data if we do things in a patch-wise manner, more than 85K samples!
However, we know from statistics that perhaps this isn't the greatest idea because there are a lot of overlap.
So we can be clever and use a training dataset with less overlap. 
However, we can create a different dataset for predictions where we reduce the strides considerably so that we take a weighted average over the predictions!

In [136]:
patches = {"x": 64, "y": 64, "t": 15}
strides = {"x": 1, "y": 1, "t": 1}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDABatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('t', 360), ('x', 200), ('y', 200)])
Patches:        OrderedDict([('t', 15), ('x', 64), ('y', 64)])
Strides:        OrderedDict([('t', 1), ('x', 1), ('y', 1)])
Num Batches:    OrderedDict([('t', 346), ('x', 137), ('y', 137)])
Dataset(size): 6494074


So this will cover use because we can take a weighted average of all of the predictions!

## Case V: Reconstructing with multiple variables

In this example, we look at how we can do reconstructions with multiple variables.
This may occur when we have used different methods to make predictions and we want to reconstruct all of them.

Another example is when we have some sort of latent variable representation and we would like to reconstruct each of the latent variable representations.


In [ ]:
import itertools

items = list(itertools.chain(*[all_batches]))
items_latent = list(itertools.chain(*[all_batches_latent]))

In [ ]:
# create items
all_batches = list(map(lambda x: x.data, xrda_batches))
items = list(itertools.chain(*[all_batches]))
items[0]

array([-1.0856306033005612, 0.9973454465835858, 0.28297849805199204,
       -1.5062947139180922, -0.5786002519685364, 1.651436537097151,
       -2.426679243393074, -0.42891262885617726, 1.2659362587055338,
       -0.8667404022651016, -0.6788861516220543, -0.09470896893689112,
       1.4913896261242878, -0.638901996684651, -0.44398195964606546,
       -0.43435127561851733, 2.205930082725455, 2.1867860889737862,
       1.004053897878877, 0.386186399174856, 0.7373685758962422,
       1.490732028150799, -0.9358338684023914, 1.1758290447821034,
       -1.2538806677490124, -0.6377515024534103, 0.9071051958003014,
       -1.4286807002259692, -0.1400687201886661, -0.8617548958596855],
      dtype=object)